In [1]:
from llama_index.core import download_loader
from pathlib import Path
from llama_index.core.node_parser import SimpleFileNodeParser, SimpleNodeParser
import os
import logging
import sys
from llama_index.core.callbacks import (
    CallbackManager
)
from llama_index.core import (SimpleDirectoryReader,
                              ServiceContext,
                              VectorStoreIndex, 
                              )
from llama_index.core.retrievers import (VectorIndexRetriever, 
                                         BaseRetriever)


from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.schema import (QueryBundle, 
                                     QueryType,
                                     NodeWithScore)
from typing import List
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.callbacks.wandb import WandbCallbackHandler
from llama_index.core import load_index_from_storage
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core import (
  VectorStoreIndex, 
  SimpleKeywordTableIndex, 
  SimpleDirectoryReader, 
  StorageContext,
  ServiceContext
 )
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1"

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/langchain/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
print("=" * 10 + "加载数据"  + "=" * 10)
"""
llamaindex通过数据连机器Reader来加载数据， 数据连接器可以加载不同数据源的数据， 
并将数据格式化为Document对象， Document对象存储文本和对应的元数据。
"""

# PDFReader = download_loader("PDFReader")
# loader = PDFReader()
# docs = loader.load_data(file=Path("./12.pdf"))
# docs = SimpleDirectoryReader("../test_data").load_data()
docs = SimpleDirectoryReader("../ops_txt_temp").load_data()
# file_path = r"/root/autodl-tmp/langchainqwen14b/llamaindex_learn/12.pdf"
# docs = SimpleDirectoryReader(input_files=[file_path]).load_data()
# print(docs)
print(len(docs))

print("=" * 10 + "分块"  + "=" * 10)
"""
将文本分割成512大小的分块来创建节点Node。
Node是llamaindex中的原子数据单位， 表示源文档的“块”
节点包含元数据以及其他节点的关系信息。
"""
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(docs)

print("=" * 10 + "开源LLM和嵌入"  + "=" * 10)
"""
使用开源模型， 对其进行量化。
通过简单地提供任务指令来生成针对任何任务（如，分类， 检索，聚类， 文本评估等）
定制的文本嵌入， 而无需任何微调，MTEB排行榜第14。
"""
def completion_to_prompt(completion):
   return f"<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{completion}<|im_end|>\n<|im_start|>assistant\n"

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|im_start|>system\n{message.content}<|im_end|>\n"
        elif message.role == "user":
            prompt += f"<|im_start|>user\n{message.content}<|im_end|>\n"
        elif message.role == "assistant":
            prompt += f"<|im_start|>assistant\n{message.content}<|im_end|>\n"

    if not prompt.startswith("<|im_start|>system"):
        prompt = "<|im_start|>system\n" + prompt

    prompt = prompt + "<|im_start|>assistant\n"

    return prompt

==========加载数据==========
40344
==========分块==========
==========开源LLM和嵌入==========


In [3]:
llm = HuggingFaceLLM(
    model_name="../model_local/qwen/Qwen1.5-7B-Chat",
    tokenizer_name="../model_local/qwen/Qwen1.5-7B-Chat",
    context_window=30000, 
    max_new_tokens=2000,
    generate_kwargs={"temperature": 0.7, 
                     "top_k": 50, 
                     "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto"
    )

embedding_model = "../embedding_model"
embed_model = HuggingFaceEmbedding(
    model_name = embedding_model
)


print("=" * 10 + "配置索引和检索器"  + "=" * 10)
"""
设置ServiceContext对象， 并使用它来构建索引和查询
索引是一种由Document对象组成的数据结构， 大模型可以使用
该索引进行查询相关上下文。
VectorStoreIndex是最常见和最易于使用的索引， 支持在大规模数据语料库上
进行检索， 包括获取top-k个最相似的节点， 并将它们传递到“响应合成”模块
"""
# ServiceContext
service_context = ServiceContext.from_defaults(
    llm=llm, 
    embed_model=embed_model
)
# index
vector_index = VectorStoreIndex(
    nodes, service_context=service_context
)

# configure retriever
retriever = VectorIndexRetriever(
    index=vector_index, 
    similarity_top_k=10, 
    service_context=service_context
)

from llama_index.postprocessor.cohere_rerank import CohereRerank
print("=" * 10 + "初始化重排序器"  + "=" * 10)

local_dir='/root/autodl-tmp/langchainqwen14b/model_local'
# Define all embeddings and rerankers
RERANKERS = {
    "WithoutReranker": "None",
    # "CohereRerank": CohereRerank(api_key=cohere_api_key, top_n=5),
    "bge-reranker-base": SentenceTransformerRerank(model=os.path.join(local_dir, "BAAI/bge-reranker-base"), 
                                                   top_n=5),
    # "bge-reranker-large": SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5)
}

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.37s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


==========配置索引和检索器==========


/tmp/ipykernel_170760/2648766769.py:29: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


==========初始化重排序器==========


In [4]:
print("=" * 10 + "检索比较"  + "=" * 10)
"""
检索器定义了在给定查询时如何有效从索引中检索相关上下文
节点后处理器：节点后处理器接收一组检索到的节点
并对它们进行转换、过滤或重排序。
节点后处理器通常在节点检索步骤之后和响应之前应用于查询引擎中
"""

# helper functions

def get_retrieved_nodes(
    query_str, reranker
):
    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    return retrieved_nodes

from IPython.display import display
from IPython.display import HTML
from copy import deepcopy
import pandas as pd

def pretty_print(df):
    # print("df \n")
    # print(df)
    # return display(HTML(df.to_html().replace("\\n", "<br>")))
    display(df)


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        node = deepcopy(node)
        # node.node.metadata = None
        node_text = node.node.get_text()
        node_text = node_text.replace("\n", " ")

        result_dict = {"Score": node.score, "Text": node_text}
        result_dicts.append(result_dict)

    # print("result_dicts \n")
    # print(result_dicts)
    pretty_print(pd.DataFrame(result_dicts))

print("=" * 10 + "检索比较"  + "=" * 10)


==========检索比较==========
==========检索比较==========


In [5]:
query_str = "如何选择合适的信令跟踪"

# Loop over rerankers
for rerank_name, reranker in RERANKERS.items():
    print(f"Running Evaluation for Reranker: {rerank_name}")

    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)
    # print("retrieved_nodes \n")
    # print(retrieved_nodes)

    #  重排序之前
    print("=" * 10 + "重排序之前"  + "=" * 10)
    visualize_retrieved_nodes(retrieved_nodes)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes
    
    print("=" * 10 + "重排序之后"  + "=" * 10)
    print(f"Visualize Retrieved Nodes for Reranker: {rerank_name}")
    visualize_retrieved_nodes(retrieved_nodes)
    


print("=" * 10 + "性能评价"  + "=" * 10)
"""
使用RetrieverEvaluator评估Retriever的质量。
定义了各种评价指标， 如命中率hit-rate何MRR， 这些指标根据每个特定问题
的ground-truth文本来评价检索结果的质量。为了简化评估数据集的构造， 
采用了合成数据生成方法。
MRR(Mean Reciprocal Rank)表示平均倒数排名， 是一个国际上通用的
对搜索算法评价的机制，并且只估计排名前10个的排名感知相关性得分。
第一个结果匹配， 分数为1， 第二个匹配分数为0.5， 第n个匹配分数为1/n。
如果没有匹配的句子， 分数为0， 最终的额分数为所有得分之和。
hit-rate衡量检索到的结果中至少包含一个与基本事实相关的项目的查询比例
或百分比。例如， 想象一个搜索引擎返回一个文档列表来表示响应用户的查询。
这里的基本事实是指该查询的已知相关文档， 通常由人类或标记数据确定。
hit-rate计算结果至少包含一个相关文档的频率。 
"""
# 构建（查询、上下文）对的评估数据集
"""
可以手动构建问题+Node id的检索评估数据集。llamaindex通过generate_question_context_pair
函数在现有文本语料库上提供合成数据集生成。
使用大模型根据每个上下文块自动生成问题
"""

# Prompt to generate questions
# qa_generate_prompt_tmpl = """\
# 上下文信息如下：

# ---------------------
# {context_str}
# ---------------------

# 给定上下文信息，不给定先验知识。
# 请根据下面的查询只生成问题。

# 你是教授. 您的任务是为即将到来的测验/考试设置{num_questions_per_chunk} 问题。
# 文件中的问题应具有多样性。问题不得包含选项，不得以 Q1/Q2 开头。问题仅限于所提供的背景信息。
# """

# Prompt to generate questions
qa_generate_prompt_tmpl = """\
Context information is below.

---------------------
{context_str}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Professor. Your task is to setup \
{num_questions_per_chunk} questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. The questions should not contain options, not start with Q1/ Q2. \
Restrict the questions to the context information provided.\
"""

from llama_index.core.evaluation import generate_question_context_pairs
qa_dataset = generate_question_context_pairs(
    nodes, llm=llm, num_questions_per_chunk=2, 
    qa_generate_prompt_tmpl=qa_generate_prompt_tmpl
)


# helper function for displaying results
def display_results(reranker_name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    # print("metric_dicts \n")
    # print(metric_dicts)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame({"Reranker": [reranker_name], 
                              "hit_rate": [hit_rate], 
                              "mrr": [mrr]})

    return metric_df

from llama_index.core.evaluation import RetrieverEvaluator

"""
 Llamaindex提供了一个函数，用于在批处理模式下对数据集运行RetrieverEvaluator。
"""

# query_str = "What are the top features of QLoRA?"


Running Evaluation for Reranker: WithoutReranker
==========重排序之前==========


Score                                               Text
0  0.517122  # 新增局向出向优先级配置(ADD OFFICESEND PRIORITY)   ## 功能...
1  0.508531  # 新增优先级拒绝策略模板配置(ADD PRIORITY REJECT POLICY PRO...
2  0.503433  | 参数作用：本参数用于配置套餐内业务的优先级。数值越大表示优先级越高。当签约业务包含在多个...
3  0.499547  通过ADD PAGINGPOLICYCFG命令增加寻呼策略配置。策略内容包括寻呼方式、寻呼消...
4  0.495534  # 修改局向出向优先级配置(SET OFFICESEND PRIORITY)   ## 功能...
5  0.492017  当选择套餐时，优先级越高会优先选择。 数据来源：本端规划。 配置原则：    * 套餐的优先...
6  0.491688  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
7  0.490952  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
8  0.488874  * 查看订单  用户可在订单管理中心查看自己提交的所有订单，可根据订单类型、订单状态、时间段...
9  0.487814  # 新增局向入向优先级配置(ADD OFFICERECV PRIORITY)   ## 功能...

==========重排序之后==========
Visualize Retrieved Nodes for Reranker: WithoutReranker


Score                                               Text
0  0.517122  # 新增局向出向优先级配置(ADD OFFICESEND PRIORITY)   ## 功能...
1  0.508531  # 新增优先级拒绝策略模板配置(ADD PRIORITY REJECT POLICY PRO...
2  0.503433  | 参数作用：本参数用于配置套餐内业务的优先级。数值越大表示优先级越高。当签约业务包含在多个...
3  0.499547  通过ADD PAGINGPOLICYCFG命令增加寻呼策略配置。策略内容包括寻呼方式、寻呼消...
4  0.495534  # 修改局向出向优先级配置(SET OFFICESEND PRIORITY)   ## 功能...
5  0.492017  当选择套餐时，优先级越高会优先选择。 数据来源：本端规划。 配置原则：    * 套餐的优先...
6  0.491688  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
7  0.490952  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
8  0.488874  * 查看订单  用户可在订单管理中心查看自己提交的所有订单，可根据订单类型、订单状态、时间段...
9  0.487814  # 新增局向入向优先级配置(ADD OFFICERECV PRIORITY)   ## 功能...

Running Evaluation for Reranker: bge-reranker-base
==========重排序之前==========


Score                                               Text
0  0.517122  # 新增局向出向优先级配置(ADD OFFICESEND PRIORITY)   ## 功能...
1  0.508531  # 新增优先级拒绝策略模板配置(ADD PRIORITY REJECT POLICY PRO...
2  0.503433  | 参数作用：本参数用于配置套餐内业务的优先级。数值越大表示优先级越高。当签约业务包含在多个...
3  0.499547  通过ADD PAGINGPOLICYCFG命令增加寻呼策略配置。策略内容包括寻呼方式、寻呼消...
4  0.495534  # 修改局向出向优先级配置(SET OFFICESEND PRIORITY)   ## 功能...
5  0.492017  当选择套餐时，优先级越高会优先选择。 数据来源：本端规划。 配置原则：    * 套餐的优先...
6  0.491688  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
7  0.490952  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
8  0.488874  * 查看订单  用户可在订单管理中心查看自己提交的所有订单，可根据订单类型、订单状态、时间段...
9  0.487814  # 新增局向入向优先级配置(ADD OFFICERECV PRIORITY)   ## 功能...

==========重排序之后==========
Visualize Retrieved Nodes for Reranker: bge-reranker-base


Score                                               Text
0  0.386399  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
1  0.378319  * PRIORITY_ONE：优先级1。按照优先级1的级别进行流控。   * PRIORIT...
2  0.014325  # 修改局向出向优先级配置(SET OFFICESEND PRIORITY)   ## 功能...
3  0.012592  通过ADD PAGINGPOLICYCFG命令增加寻呼策略配置。策略内容包括寻呼方式、寻呼消...
4  0.012458  # 新增局向出向优先级配置(ADD OFFICESEND PRIORITY)   ## 功能...

==========性能评价==========


  0%|          | 0/57107 [00:00<?, ?it/s]/root/miniconda3/envs/langchain/lib/python3.10/site-packages/llama_index/core/llama_dataset/legacy/embedding.py:99: UserWarning: Fewer questions generated (0) than requested (2).
  warnings.warn(
 16%|█▌        | 9230/57107 [24:57:02<120:02:15,  9.03s/it]

In [ ]:
for i in qa_dataset.corpus.items():
    print(i)

NameError: name 'qa_dataset' is not defined

In [ ]:
results_df = pd.DataFrame()
# Loop over rerankers
for rerank_name, reranker in RERANKERS.items():
    print(f"Running Evaluation for Reranker: {rerank_name}")

    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)
    print("retrieved_nodes")
    print(retrieved_nodes)

    if reranker != "None":
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    print("retriever_evaluator")
    print(retriever_evaluator)

    eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)
    # eval_results = retriever_evaluator.aevaluate_dataset(qa_dataset)

    print("eval_results \n")
    print(eval_results)

    current_df = display_results(rerank_name, eval_results)
    results_df = pd.concat([results_df, current_df], ignore_index=True)


Running Evaluation for Reranker: bge-reranker-base
retrieved_nodes
[NodeWithScore(node=TextNode(id_='a7e66824-1718-4ddb-9758-1670f3295a33', embedding=None, metadata={'page_label': '18', 'file_name': '12.pdf', 'file_path': '/root/autodl-tmp/langchainqwen14b/llamaindex_learn/12.pdf', 'file_type': 'application/pdf', 'file_size': 2867778, 'creation_date': '2024-06-04', 'last_modified_date': '2024-06-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ffff784c-6937-41aa-9075-47fe58c144bf', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '18', 'file_name': '12.pdf', 'file_path': '/root/autodl-tmp/langchainqwen14b/llamaindex_learn/12.pdf', 'file_type': 'application/pdf', 'file_size': 2867778, 'crea

KeyError: 'hit_rate'